# Pipeline

In this section I will ensamble all the step to create the proper Pipeline

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt
from pathlib import Path
import os
import pipeline as pp

In [13]:
# Make a list of calibration images
images = glob.glob('../test_images/*.jpg')


# select images between 0 and 7
fname = images[4]

# original
img = mpimg.imread(fname)
# plotimage(img, 0)

img_undist = pp.undistort_image(img)

# def pipeline(img):
# bitmap with streigh guides
bitmap = pp.threshold(img_undist)

# eagle view with streigh guides
img_eagle_eye = pp.eagle_eye(bitmap)

x_half = int(img_eagle_eye.shape[1] / 2)
y_half = int(img_eagle_eye.shape[0] / 2)
bottom_half = img_eagle_eye[y_half:, :]

# computing poly_fit using fit_poly in pipeline
leftx, lefty, rightx, righty = pp.find_lane_pixels_window(bottom_half, nwindows=10, margin=100, minpix=40)
left_fit, right_fit, left_fitx, right_fitx, ploty = pp.fit_poly(bottom_half.shape, leftx, lefty, rightx, righty)

# Plot the polynomial lines onto the image
left_pts = np.transpose(np.array( [left_fitx, y_half + ploty]))
right_pts = np.transpose(np.array( [right_fitx, y_half + ploty]))
lines_img = np.zeros_like(img)
thikness = 10
lines_img = cv2.polylines(lines_img, np.int32([left_pts]), False, (255, 0, 0), thikness) 
lines_img = cv2.polylines(lines_img, np.int32([right_pts]), False, (0, 255, 0), thikness) 

img_eagle_eye = pp.eagle_eye_inv(lines_img)

cv2.imshow("img", img_eagle_eye)
# computing poly_fit using fit_around_poly in pipeline
# left_fit, right_fit, left_fitx_around, right_fitx_around, ploty = pp.fit_around_poly(bottom_half, left_fit, right_fit)

# # Plot the polynomial lines onto the image
# plt.figure(1)
# plt.clf()
# plt.imshow(out_img)
# plt.plot(left_fitx_around, ploty, color='yellow')
# plt.plot(right_fitx_around, ploty, color='yellow')

In [4]:
img.shape

(720, 1280, 3)

In [5]:
img_eagle_eye.shape

(720, 1280)

In [6]:
ploty[:10]

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [7]:
pts = np.array( [left_fitx, ploty])
# pts = pts.reshape((1, 2), order='F') 

In [8]:
pts.shape

(2, 360)

In [9]:
pts.shape

(2, 360)

In [10]:
np.transpose(pts)

array([[598.16096968,   0.        ],
       [596.72976193,   1.        ],
       [595.30167021,   2.        ],
       [593.87669452,   3.        ],
       [592.45483485,   4.        ],
       [591.03609121,   5.        ],
       [589.62046359,   6.        ],
       [588.20795199,   7.        ],
       [586.79855643,   8.        ],
       [585.39227688,   9.        ],
       [583.98911336,  10.        ],
       [582.58906587,  11.        ],
       [581.1921344 ,  12.        ],
       [579.79831896,  13.        ],
       [578.40761954,  14.        ],
       [577.02003615,  15.        ],
       [575.63556878,  16.        ],
       [574.25421744,  17.        ],
       [572.87598212,  18.        ],
       [571.50086283,  19.        ],
       [570.12885956,  20.        ],
       [568.75997231,  21.        ],
       [567.3942011 ,  22.        ],
       [566.0315459 ,  23.        ],
       [564.67200674,  24.        ],
       [563.31558359,  25.        ],
       [561.96227648,  26.        ],
 